In [1]:
from torchvision.io import read_image
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
from tqdm import tqdm
import dgl
import networkx as nx
from torchvision import transforms
import torchvision.transforms.functional as F
import os
import glob
from PIL import Image

In [2]:
target_size=(256,256)
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(target_size),
    transforms.Grayscale()
])

In [4]:
# フォルダのパスを指定
folder_path = '../data/sub/car/back/*'  # フォルダのパスを適切に設定してください

# フォルダ内のファイル名を取得
file_names = glob.glob(folder_path)
print(len(file_names))
_labels=[]
# 取得したファイル名を表示
for file_name in file_names:
    _labels.append(int(file_name[-9]))
print(_labels)

546
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,

In [25]:
object_names=['car','bus','airplane']
data_file_names = []
labels = []
for i,object_name in enumerate(object_names):
    folder_path = f'../data/{object_name}/**/*'
    print(folder_path)
    file_names = glob.glob(folder_path)
    data_file_names.extend(file_names)
    labels.extend([i]*len(file_names))

print(len(data_file_names))
print(len(labels))
'''
# フォルダのパスを指定
folder_path = '../data/airplane/**/*'  # フォルダのパスを適切に設定してください

# フォルダ内のファイル名を取得
file_names = glob.glob(folder_path)
print(file_names)'''

../data/car/**/*
../data/bus/**/*
../data/airplane/**/*
1939
1939
693


"\n# フォルダのパスを指定\nfolder_path = '../data/airplane/**/*'  # フォルダのパスを適切に設定してください\n\n# フォルダ内のファイル名を取得\nfile_names = glob.glob(folder_path)\nprint(file_names)"

In [6]:
def get_nearest_neighbors(image, row, col): #最近傍ノード番号の取得
    # 画像の形状を取得
    height, width = image.shape[:2]

    # 注目画素の周囲8画素の座標を計算
    neighbors_coords = [(row-1, col-1), (row-1, col), (row-1, col+1),
                        (row, col-1), (row, col+1),
                        (row+1, col-1), (row+1, col), (row+1, col+1)]

    # 注目画素の最近傍画素の値を抜き出す
    nearest_neighbors = []
    for r, c in neighbors_coords:
        # 座標が画像範囲内かチェック
        if 0 <= r < height and 0 <= c < width:
            pixel_value = image[r, c]
            nearest_neighbors.append(pixel_value)
        else:
            # 画像範囲外の場合は0を追加するなど適切な処理を行う
            #nearest_neighbors.append(0)
            pass

    return nearest_neighbors
def make_graph(side_length): #パッチ数分のグラフを作成
    g=dgl.DGLGraph()
    g.add_nodes(side_length**2)
    square_list = np.arange(side_length**2).reshape((side_length, side_length))
    #ノード番号に対応したインデックスを取得
    inds=np.ndindex(square_list.shape)
    inds=[idx for idx in inds]
    #各ノードと最近傍ノード間にエッジを張る
    for i in range(side_length**2):
        x,y=inds[i]
        flatt_nh=get_nearest_neighbors(square_list,x,y)
        for j in flatt_nh:
            if i == j:
                continue
            else:
                g.add_edges(j,i)
    return g
def image_patch(image,num_patch):
    #画像サイズ
    size=image.shape[1]
    #1パッチ当たりの画素数
    patch_width=int(size/num_patch)
    #パッチ保存用配列
    data=[]

    for i in range(0,size,patch_width):
        for j in range(0,size,patch_width):
            data.append(image[:, i : i + patch_width, j : j + patch_width])
    
    return torch.stack(data,dim=0)

In [9]:
#パッチ数
num_patch=8

#トレーニングデータセット
graphs=[]
labels=[]
test_mode=False
complete_graph=False
test_mumber=100


for image,label in tqdm(zip(file_names,_labels)):
    if test_mode == True:
        if test_mumber < 0:
            break
        else:
            test_mumber -= 1
    image=transform(Image.open(image))
    #画像をパッチに分割
    n_feat=image_patch(image,num_patch)
    #グラフ作成
    if complete_graph:
        g=nx.complete_graph(num_patch**2)
        G=dgl.from_networkx(g)
        G.ndata['f']=n_feat
    else:
        G=make_graph(num_patch)
        #グラフにノード特徴 'f' としてパッチ画像を入力
        G.ndata['f']=n_feat

    #graphsにグラフ labelsにラベルを代入
    graphs.append(G)
    labels.append(label)

#グラフの保存
output_labels={'label':torch.tensor(labels)}
path=f'../data/ICPKGI/{num_patch}patch_gray_airplane.dgl'
dgl.save_graphs(path,g_list=graphs,labels=output_labels)



693it [01:06, 10.49it/s]
